Importer det vi trenger for geografiske analyser

In [12]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

Det vi trenger for å lage kart

In [13]:
import hvplot.pandas
from holoviews import opts
import cartopy.crs as ccrs

# støtte for scroll med mus
opts.defaults( 
    opts.Overlay(
        width=900, height=600,
        active_tools=['wheel_zoom']
))

# Last inn bomstasjoner

Hint: 
  - ls lister opp filer. 
  - Først dataframe
    - Pass på tegnsett (encoding='utf-8') og semikolon (sep=';')
  - så dataframe['geometry'] =  dataframe['Kolonne med WKT geometri'].apply(wkt.loads)
  - til sist lager du geodataframe

*NB* For bruk i hvplot må geometrikolonnen hete "geometry"


In [14]:
tmp = pd.read_csv( '45_bomstasjon-eksport.csv', sep=';', encoding='latin1')

tmp.head()

,vegobjektid,type_id,versjonid,startdato,sistmodifisert,Bomstasjonstype,Navn bompengeanlegg (fra CS),Navn bomstasjon,Vedtatt til år,Link til bomstasjon,...,vegkategori,vegstatus,vegnummer,fra hp,til hp,fra meter,til meter,stedfesting,geometri,srid
0,79786595,45,10,2019-08-02,2019-10-31T22:52:38,AutoPASS + automatisk,Ferde AS - Fv. 45 Gjesdal,Øvstabø,NaN,www.gjesdalbompengeselskap.no,...,F,V,45,1,1,10305,NaN,320617 0.16417875 0.16417875 WITH 1#2,POINT Z (8106.68059 6552587.83948 380.73347),6173
1,82443541,45,9,2019-01-10,2019-11-01T08:53:05,AutoPASS + automatisk,Ferde AS - Bomringen Bergen,Sandviken,NaN,www.ferde.no,...,E,V,39,13,13,2638,NaN,805617 0.00836412 0.00836412 WITH 1#3,POINT Z (-32178.72675 6737228.44247 31.79574),6173
2,82483423,45,8,2019-01-10,2019-10-22T22:17:53,AutoPASS + automatisk,Ferde AS - Bomringen Bergen,Gyldenpris,NaN,www.ferde.no,...,R,V,555,72,72,41120,NaN,805015 0.61284927 0.61284927 WITH 1,POINT Z (-32637.9184 6733197.69598 17.336),6173
3,82559832,45,9,2019-01-10,2019-11-01T08:53:05,AutoPASS + automatisk,Ferde AS - Bomringen Bergen,Straume,NaN,www.ferde.no,...,F,V,556,3,3,5263,NaN,805716 0.20441435 0.20441435 WITH 1#2,POINT Z (-36045.96429 6727021.84143 5.2),6173
4,82559833,45,8,2019-01-10,2019-10-31T22:52:38,AutoPASS + automatisk,Ferde AS - Bomringen Bergen,Gravdal,NaN,www.ferde.no,...,R,V,555,4,4,2638,NaN,805707 0.42898864 0.42898864 WITH 2#4,POINT Z (-35230.75777 6733476.926 15.08783),6173


In [15]:
tmp[['Geometri, punkt', 'geometri']].head(15)

,"Geometri, punkt",geometri
0,NaN,POINT Z (8106.68059 6552587.83948 380.73347)
1,NaN,POINT Z (-32178.72675 6737228.44247 31.79574)
2,NaN,POINT Z (-32637.9184 6733197.69598 17.336)
3,NaN,POINT Z (-36045.96429 6727021.84143 5.2)
4,NaN,POINT Z (-35230.75777 6733476.926 15.08783)
5,NaN,POINT Z (-33905.15497 6731397.73746 96.7)
6,NaN,POINT Z (-31572.62982 6732758.28647 10.24466)
7,NaN,POINT Z (-31178.63557 6733816.8734 15.06241)
8,NaN,POINT Z (16277.03074 6666409.91701 24.72422)
9,POINT (282139.71660693 7040113.09293828 101.19...,POINT Z (282140.17167 7040117.41042 101.2)


# Bruk _Geometri, linje_ der den finnes

Vi ønsker å hente koordinater fra egenskapen _Geometri, punkt_ der den finnes. Hvis den mangler så henter vi plassering fra _geometri_ (avledet fra plassering i nettverket). 

### Metode 1: Sett verdi basert på betingelser (For vanskelig)

Les https://datatofish.com/if-condition-in-pandas-dataframe/ for tips om hvordan du setter "subslice" til gitt verdi basert på gitte betingelser. Har funnet mange oppskrifter hvor lambda-funksjonen spiser data fra én (og bare én) kolonne, for så å gjøre ulike ting basert på denne verdien. 


Mye lærerikt, men det ene tilfellet vi trenger: **Hent fra én kolonne hvis den finnes, hvis ikke henter du fra en annen** viser seg å bli overraskende komplest. 

### Metode 2: Iterer over hver rad 
```
# tmp = pd.read_csv( '45_bomstasjon-eksport.csv', sep=';', encoding='latin1') # Har lest fra før
tmp['geometry'] = ''

for index_label, row in tmp.iterrows(): 
    
    minshapelygeom = wkt.loads( row['geometri'] ) 
    if isinstance( row['Geometri, punkt'], str ): 
        minshapelygeom = wkt.loads( row['Geometri, punkt'])
    
    tmp.at[index_label, 'geometry'] = minshapelygeom
```
Fungerer helt greit, og er ikke for vanskelig

### Metode 3: Bruk fillna() for å hente de dataverdiene som mangler fra  kolonnen der vi har data

Se under


In [16]:
# tmp = pd.read_csv( '45_bomstasjon-eksport.csv', sep=';', encoding='latin1') # Har lest fra før
tmp['Geometri, punkt'].fillna( tmp['geometri'], inplace=True)
tmp['geometry'] =  tmp['Geometri, punkt'].apply(wkt.loads)
bomstasjon = gpd.GeoDataFrame( tmp, geometry = 'geometry')
bomstasjon.crs = { 'init' : 'epsg:25833'}

In [17]:
tmp[['Geometri, punkt', 'geometri']].head(15)

,"Geometri, punkt",geometri
0,POINT Z (8106.68059 6552587.83948 380.73347),POINT Z (8106.68059 6552587.83948 380.73347)
1,POINT Z (-32178.72675 6737228.44247 31.79574),POINT Z (-32178.72675 6737228.44247 31.79574)
2,POINT Z (-32637.9184 6733197.69598 17.336),POINT Z (-32637.9184 6733197.69598 17.336)
3,POINT Z (-36045.96429 6727021.84143 5.2),POINT Z (-36045.96429 6727021.84143 5.2)
4,POINT Z (-35230.75777 6733476.926 15.08783),POINT Z (-35230.75777 6733476.926 15.08783)
5,POINT Z (-33905.15497 6731397.73746 96.7),POINT Z (-33905.15497 6731397.73746 96.7)
6,POINT Z (-31572.62982 6732758.28647 10.24466),POINT Z (-31572.62982 6732758.28647 10.24466)
7,POINT Z (-31178.63557 6733816.8734 15.06241),POINT Z (-31178.63557 6733816.8734 15.06241)
8,POINT Z (16277.03074 6666409.91701 24.72422),POINT Z (16277.03074 6666409.91701 24.72422)
9,POINT (282139.71660693 7040113.09293828 101.19...,POINT Z (282140.17167 7040117.41042 101.2)


In [18]:
bomstasjon['Navn bomstasjon'].fillna( ' ')
# ccrs.UTM(zone=33)
bomstasjon.hvplot( crs=ccrs.UTM(zone=33), hover_cols=['vegobjektid', 'Navn bomstasjon'
           'Takst liten bil', 'Takst stor bil' 'vegreferanse'], tiles='OSM',
            c='Takst liten bil', size=100   )

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Takst liten bil,vegobjektid)

In [19]:
bomstasjon.dtypes

vegobjektid                              int64
type_id                                  int64
versjonid                                int64
startdato                               object
sistmodifisert                          object
Bomstasjonstype                         object
Navn bompengeanlegg (fra CS)            object
Navn bomstasjon                         object
Vedtatt til år                         float64
Link til bomstasjon                     object
Innkrevningsretning                     object
Bomstasjon_Id                          float64
Bompengeanlegg_Id                      float64
Takst liten bil                        float64
Takst stor bil                         float64
Tidsdifferensiert takst                 object
Rushtidstakst liten bil                float64
Rushtidstakst stor bil                 float64
Rushtid morgen, fra                     object
Rushtid morgen, til                     object
Rushtid ettermiddag, fra                object
Rushtid etter

In [20]:
bomstasjon['antall'] = 1

bomstasjon[['Navn bompengeanlegg (fra CS)', 'antall']].groupby( 'Navn bompengeanlegg (fra CS)').sum().sort_values('antall', 
                                            ascending=False)

,antall
Navn bompengeanlegg (fra CS),
Fjellinjen AS,90
Ferde AS - Bymiljøpakken Nord Jæren,38
Vegamot AS,33
Ferde AS - Bomringen Bergen,14
Vegfinans Bypakke Grenland AS,13
Ferde AS - Haugalandspakken,13
Veipakke Salten AS,10
Østfold Bompengeselskap AS,10
Vegfinans E18 Vestfold AS,7


In [21]:
pd.pivot_table( bomstasjon, values=['Takst liten bil', 'Takst stor bil'], index=['Navn bompengeanlegg (fra CS)', 'Navn bomstasjon'] )

Takst liten bil  \
Navn bompengeanlegg (fra CS)           Navn bomstasjon                                        
BPS Nord - Helgeland Veiutvikling AS   Fv. 78 Drevja                                   30.0   
                                       Fv. 78 Hjartåsen bomstasjon Kulstad             26.0   
                                       Fv. 78 Toven                                    91.0   
BPS Nord - Hålogalandsbrua AS          E6/E10 Leirvik                                  42.0   
BPS Nord - Nordland Bompengeselskap AS E6 Reinforshei                                  18.0   
                                       E6 Skamdal                                      13.0   
                                       Forsmo - eksisterende E6                        26.0   
                                       Grane - Vefsn                                   31.0   
                                       Svenningvatnet                                  31.0   
Bergen ytre bomring                    Åsamyrane                                       30.0   
Eksportvegen AS                        Skjegstad bomstasjon                            35.0   
                                       Skorgen bomstasjon                              35.0   
                                       Vikebukt bomstasjon                             69.0   
                                       Vågstranda bomstasjon                           33.0   
FOSENVEGENE EI TIM TE  BYN AS          Krinsvatn                                       84.0   
Ferde AS - Askøypakken                 Nedre Kleppe                                    29.0   
                                       Stongafjellet                                   29.0   
                                       Storeklubben                                    29.0   
                                       Vatnavatnet                                     29.0   
Ferde AS - Bomringen Bergen            Damsgårdsveien                                  30.0   
                                       Fjøsangerveien                                  30.0   
                                       Florida                                         30.0   
                                       Fyllingen                                       30.0   
                                       Gravdal                                         30.0   
                                       Gyldenpris                                      30.0   
                                       Kalfaret                                        30.0   
                                       Michael-Krohnsgt                                30.0   
                                       Natlandsveien                                   30.0   
                                       Nyebroen                                        30.0   
                                       Ringveg Vest Dolviken                           30.0   
...                                                                                     ...   
Vegfinans E6 Ringebu-Otta AS           Fv. 312 Hesteskobakken                          16.0   
                                       Fv. 312 Moastugu                                11.0   
Vegfinans Fv33 Oppland AS              Fv. 33 Langsletta                               34.0   
Vegfinans Fv34 Oppland AS              Fv34 Grime                                      33.0   
Vegfinans Gausdalsvegen AS             Fv.255 Fåberg                                   15.0   
                                       Fv.315 Jørstadmoen                              15.0   
Vegfinans Hallingporten AS             Rv. 7 Brekkebygda                               78.0   
Vegfinans Rv36 Telemark AS             Rv. 36 Ullevik                                  28.0   
Vegfinans Rv4 Oppland AS               Fv. 16 Dynna                                    43.0   
                                       Rv. 4 Dynna                                    